## 1. Introduction

## 1.1 Background of the problem

Nairobi is the capital city of Kenya and accounts for 40 percent of all financial transactions in the country.The 
population of Nairobi as at 2019 was estimated to be 4,397,073 accounting for more than 10 percent of the total Kenyan populationThis makes
it a viable option for an investor looking to get higher margins especially in the entertainment and hospitality industry.
For an **entrepreuner looking to set up a major restaurant** in the city strategic location is key to the success and sustainability of the business , this is primarily because customers are the backbone of such a business and the more the customers the heigher the profit margins .
Owing to the number of existing restaurants within the city it is important to set up aa new one where it would be much more reasonable to target newer audiences . 

However, it is increasingly  difficult to obtain information that will guide investors in this direction. 

In regard to this I aim to use Kernel Density Estimates to help solve this through weighting the distances of all the data points of the existing restaurants  from the center of the city . USing this a map can be generated and the points with higher concentration of restaurants identified

## 2. Data Description


Obtaining relevant location data is key to solving the problem of strategic location of a restaurant .
I have therefore used geolocation data from **Foursquare** to get Data partaining the location of the restaurant and its neigbours within a 4 km  radius . This include the longitude and latitude coordinates as these are essentiall in mapping ,the shop name of the restaurant and the about has also been collected . This I have then converted to a pandas dataframe for ease of use .

With the infomation about the location of the existing restaurants it is possible to apply KDE(Kernel Density Estimation) to identify the concentration of exisitng restaurants and set up a new restaurant where there exists few

In [8]:
## making api calls to foursquare
import pandas as pd
import requests as rq
CLIENT_ID = 'DPIZ13PIHP1BYV3EAJVTRO4GPMFPOCBVTHKT45OPNYCKCWGS' # my Foursquare ID
CLIENT_SECRET = 'MMKATV3MMUCFBLIQ4MJELX5ZDD12AFCMCHKJWNI223NRHCIL' #my Foursquare Secret
VERSION = '20180604'
LIMIT = 30
request_data = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section":"restaurant",
    "near": "Nairobi",
    "radius": 4000,#within 4km radius
    "limit": 100}

df_nai = rq.get("https://api.foursquare.com/v2/venues/explore", params=request_data)

d = df_nai.json()["response"]
items = d["groups"][0]["items"]
df_restaurant = []
for item in items:
    venue = item["venue"]
    categories, uid, name,location = venue["categories"], venue["id"], venue["name"],item["venue"]["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (name, shortname, lat, lng)
    df_restaurant.append(datarow)
df = pd.DataFrame(df_restaurant, columns=["name", "shortname", "lat", "lng"])
df.head()

,name,shortname,lat,lng
0,CJ's,Coffee Shop,-1.283147,36.818323
1,"Subway,Kenyatta Avenue",Sandwiches,-1.284390,36.821827
2,Cafe Maghreb (Serena),Eastern European,-1.286752,36.814836
3,Najmi Fast Food,Fast Food,-1.280862,36.821209
4,Tatu Restaurant,African,-1.278291,36.816383


## 3. Methodology

The Kernel Density tool calculates the density of features in a neighborhood around those features. It can be calculated for both point and line features.
It can be useful in calculation of point features through
> Kernel Density calculates the density of point features around each output raster cell 

> This is done with the following procedure
>
a smoothly curved surface is fitted over each point. The surface value is highest at the location of the point and diminishes with increasing distance from the point, reaching zero at the Search radius distance from the point. Only a circular neighborhood is possible. The volume under the surface equals the Population field value for the point, or 1 if NONE is specified. The density at each output raster cell is calculated by adding the values of all the kernel surfaces where they overlay the raster cell center


The predicted density at a new (x,y) location is determined by the following formula
$Density=\frac{1}{radius^2}=\sum_{i=n}^{n}\left[\left(\frac{3}{\pi}\left(1-\frac{dist_i}{radius_i}\right)^2\right)^2\right]$

where i=1,...,n are the input points. Only include points in the sum if they are within the radius distance of the (x,y) location

$dist_i$ is the distance between point i and the (x,y) location.

The formula will need to be calculated for every location where we want to estimate the density. Since a raster is being created, the calculations are applied to the center of every cell in the output raster.

## 4. Results

Heatmap-based kernel density estimation is used. Heatmat is an already 
 implemented as plugin for Folium, which can be used
to visualize data to map

In [11]:

Nairobi_center = d["geocode"]["center"]
Nairobi_center
import folium
from folium import plugins
# create map of Nairobii using latitude and longitude values
map_nairobi = folium.Map(location=[Nairobi_center["lat"], Nairobi_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='red',
            fill_opacity=0.7,
            parse_html=False).add_to(map_nairobi)

add_markers(df)
hm_data = df[["lat", "lng"]].as_matrix().tolist()
map_nairobi.add_child(plugins.HeatMap(hm_data))

map_nairobi

C:\Users\christopher\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


## 5. Discussion

In [5]:
lat = -1.28333
lng = 36.81667
map_NAI = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="MAJOR_RESTAURANT_BEST_PLACE",
    color='red',
    fill=True,
    fill_color='yellow',
    fill_opacity=0.7,
    parse_html=False).add_to(map_NAI)
map_NAI

 Based on the estimated it would be appropriate to set up along Utalii Street

## 6. Conclusion


Strategic location of a business with customers at its core is important and a restaurant being one such it is therefore appropriate to use good methods for visualizing and communicating this to potential investors 